In [6]:
file_path = r"C:\Users\BXDM\Downloads\601899历史数据.csv"

In [7]:
import pandas as pd

# 加载CSV文件
data = pd.read_csv(file_path)

# 将日期列转换为日期格式
data['日期'] = pd.to_datetime(data['日期'])

# 提取年份和月份
data['年'] = data['日期'].dt.year
data['月'] = data['日期'].dt.month

# 检查年初和年末的交易日期
annual_returns = data.groupby('年').agg(
    年初日期=('日期', 'min'),
    年末日期=('日期', 'max')
).reset_index()

# 合并收盘价到年初和年末日期，获取年初和年末收盘价
annual_returns = annual_returns.merge(
    data[['日期', '收盘']], left_on='年初日期', right_on='日期', how='left'
).rename(columns={'收盘': '年初收盘价'}).drop(columns=['日期'])

annual_returns = annual_returns.merge(
    data[['日期', '收盘']], left_on='年末日期', right_on='日期', how='left'
).rename(columns={'收盘': '年末收盘价'}).drop(columns=['日期'])

# 计算年度收益率（不乘以100，保持比例形式）
annual_returns['年度收益率(%)'] = (annual_returns['年末收盘价'] - annual_returns['年初收盘价']) / annual_returns['年初收盘价']

# 检查月初和月末的交易日期
monthly_returns = data.groupby(['年', '月']).agg(
    月初日期=('日期', 'min'),
    月末日期=('日期', 'max')
).reset_index()

# 合并收盘价到月初和月末日期，获取月初和月末收盘价
monthly_returns = monthly_returns.merge(
    data[['日期', '收盘']], left_on='月初日期', right_on='日期', how='left'
).rename(columns={'收盘': '月初收盘价'}).drop(columns=['日期'])

monthly_returns = monthly_returns.merge(
    data[['日期', '收盘']], left_on='月末日期', right_on='日期', how='left'
).rename(columns={'收盘': '月末收盘价'}).drop(columns=['日期'])

# 计算每月收益率（不乘以100，保持比例形式）
monthly_returns['收益率'] = (monthly_returns['月末收盘价'] - monthly_returns['月初收盘价']) / monthly_returns['月初收盘价']

# 创建一个以年为行，月为列的透视表
pivot_table = monthly_returns.pivot(index='年', columns='月', values='收益率')

# 将年度收益率合并到透视表中
pivot_table['年度收益率'] = annual_returns.set_index('年')['年度收益率(%)']

In [8]:
pivot_table

月,1,2,3,4,5,6,7,8,9,10,11,12,年度收益率
年,,,,,,,,,,,,,
2020,-0.096842,0.077720,-0.141860,0.048257,-0.045455,0.131105,0.274510,0.231164,-0.162125,0.123794,0.292072,-0.010650,0.955789
2021,-0.043825,0.167625,-0.219156,0.132444,-0.026863,-0.152972,0.046987,0.027752,-0.056127,0.019493,-0.031489,-0.043393,-0.033865
2022,-0.006110,0.094634,0.017040,-0.033188,-0.071956,-0.053753,-0.057022,-0.004525,-0.093642,0.019305,0.198780,-0.029126,0.018330
2023,0.119658,-0.070090,0.034224,0.058581,-0.173241,0.028959,0.090528,-0.052874,-0.052344,0.019672,-0.001639,0.034884,0.183286
2024,-0.036029,0.080791,0.269434,0.043865,-0.006279,0.013849,-0.080931,-0.037883,0.162076,-0.063474,-0.009275,NaN,0.368295


In [9]:
data

,日期,收盘,开盘,高,低,交易量,涨跌幅,年,月
0,2024-11-07,17.09,16.72,17.10,16.50,280.88M,-1.27%,2024,11
1,2024-11-06,17.31,17.65,17.70,17.16,193.33M,-2.20%,2024,11
2,2024-11-05,17.70,17.26,17.72,17.15,185.52M,2.02%,2024,11
3,2024-11-04,17.35,17.36,17.42,16.97,150.86M,0.58%,2024,11
4,2024-11-01,17.25,16.72,17.50,16.64,251.43M,2.56%,2024,11
...,...,...,...,...,...,...,...,...,...
1169,2020-01-08,4.70,4.79,4.83,4.61,440.34M,0.64%,2020,1
1170,2020-01-07,4.67,4.73,4.73,4.62,402.40M,-3.91%,2020,1
1171,2020-01-06,4.86,4.94,5.02,4.65,577.60M,1.25%,2020,1
1172,2020-01-03,4.80,4.78,4.90,4.71,274.10M,1.05%,2020,1


In [10]:
pivot_table

月,1,2,3,4,5,6,7,8,9,10,11,12,年度收益率
年,,,,,,,,,,,,,
2020,-0.096842,0.077720,-0.141860,0.048257,-0.045455,0.131105,0.274510,0.231164,-0.162125,0.123794,0.292072,-0.010650,0.955789
2021,-0.043825,0.167625,-0.219156,0.132444,-0.026863,-0.152972,0.046987,0.027752,-0.056127,0.019493,-0.031489,-0.043393,-0.033865
2022,-0.006110,0.094634,0.017040,-0.033188,-0.071956,-0.053753,-0.057022,-0.004525,-0.093642,0.019305,0.198780,-0.029126,0.018330
2023,0.119658,-0.070090,0.034224,0.058581,-0.173241,0.028959,0.090528,-0.052874,-0.052344,0.019672,-0.001639,0.034884,0.183286
2024,-0.036029,0.080791,0.269434,0.043865,-0.006279,0.013849,-0.080931,-0.037883,0.162076,-0.063474,-0.009275,NaN,0.368295


In [12]:
pivot_table.to_excel('ac.xlsx')

In [5]:
# 计算每年的年度收益率，年度收益率为每年最后一个交易日的收盘价与第一个交易日的收盘价之间的变化百分比

# 计算年初和年末的收盘价
annual_returns = data.groupby('年').agg(
    年初收盘价=('收盘', 'first'),
    年末收盘价=('收盘', 'last')
).reset_index()

# 计算年度收益率（年末收盘价 - 年初收盘价）/ 年初收盘价 * 100%
annual_returns['年度收益率(%)'] = ((annual_returns['年末收盘价'] - annual_returns['年初收盘价']) / annual_returns['年初收盘价']) * 100
annual_returns

,年,年初收盘价,年末收盘价,年度收益率(%)
0,2020,4.92,5.09,3.455285
1,2021,8.30,4.95,-40.361446
2,2022,6.00,8.16,36.000000
3,2023,7.50,6.09,-18.800000
4,2024,9.99,7.68,-23.123123
